In [7]:
from traffic.core import Traffic
from preprocessing import get_complete_flights, preprocess_traffic, generate_dummy_columns
import h5py
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import itertools

In [2]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.10.0
Num GPUs Available:  1


In [3]:
def seconds_till_arrival(flights_data: pd.DataFrame):
    time_till_arrival = flights_data["arrival_time"]- flights_data["timestamp"]
    seconds = time_till_arrival.dt.total_seconds()
    return seconds

In [4]:
def build_sequential(lr, input_dims, output_dims, layerSizes, activation, loss):
    model = keras.Sequential()
    model.add (keras.layers.Input(shape=input_dims))
    for size in layerSizes:
        model.add(keras.layers.Dense(size))
        model.add(keras.layers.LeakyReLU(alpha=0.05))

    model.add(keras.layers.Dense(output_dims,activation=activation))

    model.compile(optimizer=Adam(learning_rate=lr), loss=loss)

    return model

In [5]:
scaler = scaler = load('../trained_models/std_scaler.bin')
file = "../data/Frankfurt_LH_2301.h5"

with h5py.File(file, 'r') as f:
    first = True
    i = 0
    for key in list(f.keys()):
        if i ==3:
            break
        print(key)
        try:
            flights = Traffic.from_file(file, key=key,
                                    parse_dates=["day", "firstseen", "hour", "last_position",
                                                 "lastseen", "timestamp"])
        except:
            continue

        try:
            df_flights = preprocess_traffic(flights)
        except AttributeError:
            continue

        df = df_flights[
            ["distance", "altitude", "geoaltitude", "arrival_time", "timestamp", "vertical_rate",
             "groundspeed", "month", "weekday", "holiday"]].dropna()

        if not first:
            df_eval = pd.concat([df, df_eval])
        else:
            df_eval = df
            first = False
        i+=1

LH_230101
LH_230103
LH_230104
LH_230105


In [6]:
df_eval.head()

,distance,altitude,geoaltitude,arrival_time,timestamp,vertical_rate,groundspeed,month,weekday,holiday
6,6211.996843,2025.0,1825.0,2023-01-05 06:29:28+00:00,2023-01-05 00:00:01+00:00,1024.0,200.0,1,3,False
12,6211.996843,2025.0,1825.0,2023-01-05 06:29:28+00:00,2023-01-05 00:00:02+00:00,896.0,201.0,1,3,False
17,6211.996843,2050.0,1825.0,2023-01-05 06:29:28+00:00,2023-01-05 00:00:03+00:00,704.0,203.0,1,3,False
25,6211.996843,2050.0,1850.0,2023-01-05 06:29:28+00:00,2023-01-05 00:00:04+00:00,640.0,204.0,1,3,False
33,6212.805560,2050.0,1850.0,2023-01-05 06:29:28+00:00,2023-01-05 00:00:05+00:00,768.0,205.0,1,3,False


In [8]:
df_eval = df_eval.sample(frac = 1)
X = np.array(df_eval[["distance", "altitude", "geoaltitude", "vertical_rate",
                       "groundspeed"]])  # we only take the feature
y = seconds_till_arrival(df_eval)
X_scaled = scaler.transform(X)

In [9]:
df_eval.shape

(23582198, 7)

In [10]:
import os
model_file = ".." + os.sep + "data" + os.sep + "model_large"

In [11]:
model = load_model(model_file)
model.summary()

C:\Users\dario\anaconda3\envs\Lufthansa-Arrival-Time-Prediction\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              6144      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 2

In [13]:
model.evaluate(X_scaled,y)

 81925/736944 [==>...........................] - ETA: 19:44 - loss: 205310.9688

KeyboardInterrupt: 

sadf